# Quantum Max Flow

Having experimented with the max flow algorithm in Notebook 1 using Google's `ortools` library, we will now attempt to apply this algorithm to the quantum world.

For this purpose, we will use DWave.

In [4]:
# Load Data
import geopandas as gpd

gdf_edges = gpd.read_file("../data/gdf_edges_bi.geojson") # Read bi-directional edges
gdf_nodes = gpd.read_file("../data/gdf_nodes.geojson") # Read nodes

In [11]:
from dimod import ExactSolver, BinaryQuadraticModel

# Create a simple BQM (Binary Quadratic Model)
bqm = BinaryQuadraticModel({'x': -1, 'y': -1}, {('x', 'y'): 2}, 0.0, 'BINARY')

# Use local classical solver
sampler = ExactSolver()
sampleset = sampler.sample(bqm)

print(sampleset)

   x  y energy num_oc.
1  1  0   -1.0       1
3  0  1   -1.0       1
0  0  0    0.0       1
2  1  1    0.0       1
['BINARY', 4 rows, 4 samples, 2 variables]
